In [0]:
# imports
import os

import pandas as pd

In [0]:
VOLUME_ROOT_PATH = "/Volumes/cscie103_catalog/final_project/data"
# place where raw csvs land after download
VOLUME_TARGET_DIR = f"{VOLUME_ROOT_PATH}/raw"
# raw data
VOLUME_BRONZE_DIR = f"{VOLUME_ROOT_PATH}/bronze"
# place where prepared data is written
VOLUME_SILVER_DIR = f"{VOLUME_ROOT_PATH}/silver"
# place where final data is written
VOLUME_GOLD_DIR = f"{VOLUME_ROOT_PATH}/gold"

# ensure all paths exist
for path in [VOLUME_TARGET_DIR, VOLUME_BRONZE_DIR, VOLUME_SILVER_DIR, VOLUME_GOLD_DIR]:
  if not os.path.exists(path):
    os.makedirs(path, exist_ok=True)

In [0]:
# read train_df from VOLUME_SILVER_DIR/encoded_train_df
data_df = spark.read.parquet(VOLUME_SILVER_DIR + "/encoded_train_df")

data_df.printSchema()
print(f'data_df has {data_df.count()} rows and {len(data_df.columns)} columns\n')

In [0]:
# Checking rule of thumb: N > 50 + 8 * k, where N - number of rows, k - number of columns
print(f'N = {data_df.count()}, k = {len(data_df.columns)}')
print(f'N > 50 + 8 * k is {data_df.count() > 50 + 8 * len(data_df.columns)} ({50 + 8 * len(data_df.columns)})')

In [0]:
display(data_df)

In [0]:
# Train / Test split
# time series data, cannot perform random split
# let's do it for hardcoded year of 2016

# train_df - all rows with is_year__2016 = 0 & is_year__2017 = 0
# test_df - all rows with is_year__2016 = 1 & is_year__2017 = 0
train_df = data_df.filter((data_df['is_year__2016'] == 0) & (data_df['is_year__2017'] == 0))
test_df = data_df.filter((data_df['is_year__2016'] == 1) | (data_df['is_year__2017'] == 1))

X_train = train_df.drop('total_daily_sales')
y_train = train_df.select('total_daily_sales')
X_test = test_df.drop('total_daily_sales')
y_test = test_df.select('total_daily_sales')

print(f'train_df has {X_train.count()} rows and {len(X_train.columns)} columns')
print(f'test_df has {X_test.count()} rows and {len(X_test.columns)} columns')

In [0]:
# to pandas
X_train = X_train.toPandas()
X_test = X_test.toPandas()
y_train = y_train.toPandas()
y_test = y_test.toPandas()

## WIP (does not quite work)

In [0]:
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

In [0]:
y_pred = lr_model.predict(X_test).ravel()
y_fit = lr_model.predict(X_train).ravel()

y_pred = pd.Series(y_pred, index = y_test.index)
y_fit = pd.Series(y_fit, index = y_train.index)

In [0]:
# rmsle
def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

print(f'train rmsle = {rmsle(y_train, y_fit)}')
print(f'test rmsle = {rmsle(y_test, y_pred)}')